In [2]:
# seas1415 = Season ('2015')
seas1516 = Season('2016')
# for seas in (seas1516):
for event in seas1516.events:
    if event.name == 'gpfra2015':
        print 'GP France 2015 was cancelled partway through.'
        continue
    event.get_event_info()

ISU GP 2015 Progressive Skate America
ISU GP Skate Canada International 2015
ISU GP Audi Cup of China 2015
ISU GP Rostelecom Cup 2015
GP France 2015 was cancelled partway through.
ISU GP NHK Trophy 2015
ISU Grand Prix of Figure Skating Final
ISU European Figure Skating Championships 2016
ISU Four Continents Figure Skating Championships
ISU World Figure Skating Championships 2016


In [3]:
nhk15 = seas1516.events[5]
nhk15_men = nhk15.disciplines[0]
nhk15_men_short = seas1516.events[5].disciplines[0].segments[0]
nhk15_men_short.parse_raw_csv()

In [4]:
yuzu = nhk15_men_short.scorecards[0]
# yuzu.print_scorecard()
kovtun = nhk15_men_short.scorecards[3]
yuzu.print_scorecard()

Skater(name='Yuzuru HANYU', country='JPN', discipline=<DisciplineType.men: 0>) 1 start: 12
TES: 59.44, PCS: 46.89, Total: 106.33
Deductions: {}
Element(number=1, name='4S', info='', base_value=10.5, bonus=True, goe=1.0, goes=[1, 2, 1, 1, 1, 2, 0, 0, 1], points=11.5)
Element(number=2, name='4T+3T', info='', base_value=14.6, bonus=True, goe=2.57, goes=[2, 3, 2, 3, 3, 3, 2, 2, 3], points=17.17)
Element(number=3, name='FCSp4', info='', base_value=3.2, bonus=True, goe=1.07, goes=[2, 3, 2, 2, 3, 2, 2, 2, 2], points=4.27)
Element(number=4, name='3A', info='', base_value=9.35, bonus=True, goe=2.43, goes=[2, 3, 2, 2, 3, 3, 3, 2, 2], points=11.78)
Element(number=5, name='CSSp4', info='', base_value=3.0, bonus=True, goe=1.21, goes=[3, 2, 3, 2, 3, 2, 3, 2, 2], points=4.21)
Element(number=6, name='StSq4', info='', base_value=3.9, bonus=True, goe=1.9, goes=[3, 3, 2, 3, 3, 3, 2, 3, 2], points=5.8)
Element(number=7, name='CCoSp3p4', info='', base_value=3.5, bonus=True, goe=1.21, goes=[3, 2, 3, 2, 3, 3

In [1]:
import requests

from model.Discipline import Discipline, DisciplineType
from model.Segment import SegmentType
from model.Season import Season

In [2]:
sochi = Season('2014')

In [3]:
usa = sochi.events[0]
usa.get_event_info()

NameError: global name 'url' is not defined

In [4]:
page = requests.get(sochi.events[0].url)

In [5]:
url = sochi.events[0].url